In [ ]:
import pandas as pd
import altair as alt
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'

## <font color="#747a7f">6.3</font> Consultas Dinâmicas

*Consultas dinâmicas* permitem uma exploração rápida e reversível dos dados para isolar padrões de interesse. Conforme definido por [Ahlberg, Williamson e Shneiderman](https://www.cs.umd.edu/~ben/papers/Ahlberg1992Dynamic.pdf), uma consulta dinâmica:

- representa uma consulta graficamente,
- oferece limites visíveis no intervalo da consulta,
- oferece uma representação gráfica dos dados e do resultado da consulta,
- fornece feedback imediato do resultado após cada ajuste da consulta,
- e permite que usuários iniciantes comecem a trabalhar com pouco treinamento.

Uma abordagem comum é modificar os parâmetros da consulta usando widgets de interface do usuário padrões, como controles deslizantes, botões do tipo radio e menus drop-down. Para gerar widgets de consulta dinâmica, nós podemos aplicar uma operação `bind` da seleção a um ou mais campos que desejamos fazer uma consulta.

Vamos construir um scatter plot interativo que usa uma consulta dinâmica para filtrar a exibição. Dado um scatter plot de classificações de filmes (do Rotten Tomatoes e IMDB), nós podemos adicionar uma seleção sobre o campo `Major_Genre` para permitir uma filtragem interativa por gênero do filme.

Para começar, vamos extrair os gêneros únicos (não nulos) dos dados em `movies`:

In [ ]:
df = pd.read_json(movies) # Carrega os dados dos filmes
genres = df['Major_Genre'].unique() # retorna valores de campos únicos
genres = list(filter(lambda d: d is not None, genres)) # filtra valores Nulos (None)
genres.sort() # ordena alfabeticamente

Para uso posterior, vamos também definir uma lista de valores únicos de `MPAA_Rating`:

In [ ]:
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

Agora vamos criar um limite de seleção `point` para um menu drop-down.

*Use o menu de consulta dinâmica abaixo para explorar os dados. Como as avaliações variam por gênero? Como você modificaria o código para filtrar por `MPAA_Rating` (G, PG, PG-13, etc.) ao invés de `Major_Genre`?*

In [ ]:
selectGenre = alt.selection_point(
    name='Select', # nomeia a seleção como 'Select'
    fields=['Major_Genre'], # limita a seleção ao campo Major_Genre
    value=genres[0], # usa a primeira entrada de gênero como valor inicial
    bind=alt.binding_select(options=genres) # vincula a um menu de valores únicos de gênero
)

alt.Chart(movies).mark_circle().add_params(
    selectGenre
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selectGenre, alt.value(0.75), alt.value(0.05))
)

alt.Chart(...)

Nossa construção acima aproveita vários aspectos das seleções:

- Nós demos à seleção um nome (`'Select'`). Este nome não é obrigatório, mas nos permite influenciar o texto no rótulo do menu de consulta dinâmica gerado. (*O que acontece se você remover o nome? Teste!*)
- Nós restringimos a seleção a um campo específico (`Major_Genre`). Anteriormente, quando nós usamos uma seleção `point`, a seleção mapeou pontos individuais. Ao limitar a seleção a um campo específico, nós podemos selecionar *todos* os pontos cujo valor no campo `Major_Genre` corresponda ao valor selecionado.
- Nós inicializamos a seleção com um valor inicial usando `value=....`
- Nós vinculamos a seleção, com o `bind`, a um widget de interface, neste caso um menu drop-down usando o `binding_select`.
- Como antes, nós usamos ainda uma codificação condicional para controlar o canal de opacidade.

### <font color="#747a7f">6.3.1</font> Vinculando Seleções a Múltiplas Entradas

Uma única instância de seleção pode ser limite para *múltiplos* widgets de consulta dinâmica. Vamos modificar o exemplo acima para oferecer filtros para ambos, `Major_Genre` e `MPAA_Rating`, usando botões do tipo radio ao invés de um menu. Nossa seleção `point` agora é definida por um único *par* de valores de gênero e classificação MPAA.

*Procure combinações inesperadas de gênero e classificação. Há algum filme de terror classificado como G ou PG?*

In [ ]:
# seleção de valor único sobre pares [Major_Genre, MPAA_Rating]
# usa valores específicos predefinidos como os valores iniciais selecionados
selection = alt.selection_point(
    name='Select',
    fields=['Major_Genre', 'MPAA_Rating'],
    value=[{'Major_Genre': 'Drama', 'MPAA_Rating': 'R'}],
    bind={
        'Major_Genre': alt.binding_select(options=genres),
        'MPAA_Rating': alt.binding_radio(options=mpaa)
    }
)

# scatter plot, modifica a opacidade com base na seleção
alt.Chart(movies).mark_circle().add_params(
    selection
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
)


alt.Chart(...)

*Curiosidades: A classificação PG-13 não existia quando os filmes [Tubarão](https://www.imdb.com/title/tt0073195/) e [Tubarão 2](https://www.imdb.com/title/tt0077766/) foram lançados. O primeiro filme a receber a classificação PG-13 foi [Amanhecer Violento](https://www.imdb.com/title/tt0087985/), de 1984.*

### <font color="#747a7f">6.3.2</font> Usando Visualizações como Consultas Dinâmicas

Embora os widgets de interface padrões mostrem os *possíveis* valores dos parâmetros da consulta, eles não visualizam a *distribuição* desses valores. Nós também podemos desejar usar interações mais ricas, como seleções de múltiplos valores ou de intervalos, ao invés de widgets de entrada que selecionam apenas um valor por vez.

Para resolver estes problemas, nós podemos criar gráficos adicionais para visualizar os dados e suportar consultas dinâmicas. Vamos adicionar um histograma da contagem de filmes por ano e usar uma seleção de intervalo para destacar dinamicamente os filmes em períodos de tempo selecionados.

*Interaja com o histograma de anos para explorar filmes de diferentes períodos. Você vê algum indício de [viés de amostragem](https://en.wikipedia.org/wiki/Sampling_bias) ao longo dos anos? (Como os anos e as classificações dos críticos se relacionam?)*

*Os anos variam de 1930 a 2040! Os filmes futuros estão em pré-produção, ou há erros do tipo "erro por um século"? Além disso, dependendo de qual fuso horário você está, você pode ver um pequeno pico em 1969 ou 1970. Por que isso acontece? (Veja o final do notebook para uma explicação!)*

In [ ]:
brush = alt.selection_interval(
    encodings=['x'] # limita a seleção aos valores do eixo x (ano)
)

# histograma de consulta dinâmica
years = alt.Chart(movies).mark_bar().add_params(
    brush
).encode(
    alt.X('year(Release_Date):T', title='Filmes por Ano de Lançamento'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
)

# scatter plot, modifica a opacidade com base na seleção
ratings = alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

alt.vconcat(years, ratings).properties(spacing=5)


alt.VConcatChart(...)

O exemplo acima oferece consultas dinâmicas usando uma *seleção vinculada* entre gráficos:

- Nós criamos uma seleção `interval` (`brush`) e definimos `encodings=['x']` para limitar a seleção apenas ao eixo x, resultando em um intervalo de seleção unidimensional.
- Nós registramos o `brush` com nosso histograma de filmes por ano por meio do `.add_params(brush)`.
- Nós usamos o `brush` em uma codificação condicional para ajustar a opacidade do scatter plot com o parâmetro `opacity`.

Esta técnica de interação de selecionar elementos em um gráfico e ver destaques vinculados em um ou mais outros gráficos é conhecida como [brushing & linking](https://en.wikipedia.org/wiki/Brushing_and_linking).

## <font color="#747a7f">6.4</font> Panning & Zooming

O scatter plot das classificações dos filmes está um pouco congestionado em alguns pontos, o que dificulta a análise das regiões mais densas. Usando as técnicas de interação *panning* (deslocamento) e *zooming* (zoom), nós podemos inspecionar as regiões densas de forma mais detalhada.

Vamos começar pensando em como poderíamos expressar deslocamento e zoom usando seleções do Altair. O que define a "área de visualização" de um gráfico? *Os domínios das escalas dos eixos!*

Nós podemos alterar os domínios das escalas para modificar o intervalo visualizado dos valores dos dados. Para fazer isso interativamente, nós podemos vincular uma seleção `interval` aos domínios das escalas com o código `bind='scales'`. O resultado é que, ao invés de um intervalo brush que nós podemos arrastar e dar zoom, nós podemos arrastar e dar zoom em toda a área do gráfico!

*No gráfico abaixo, clique e arraste para mover a visualização ou use o scroll do mouse para dar zoom (ajustar a escala) na visualização. O que você pode descobrir sobre a precisão dos valores de classificação oferecidos?*

In [ ]:
alt.Chart(movies).mark_circle().add_params(
    alt.selection_interval(bind='scales')
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # usa o parâmetro minExtent para estabilizar o título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

alt.Chart(...)

*Ao dar zoom, nós podemos ver que os valores das classificações têm precisão limitada! As classificações do Rotten Tomatoes são inteiros, enquanto as classificações do IMDB são truncadas para décimos. Como resultado, há sobreposição de pontos mesmo quando damos zoom, com vários filmes compartilhando os mesmos valores de classificação.*

Lendo o código acima, você pode notar o código `alt.Axis(minExtent=30)` no canal de codificação `y`. O parâmetro `minExtent` garante que um espaço mínimo seja reservado para os marcadores e rótulos dos eixos. Por que fazer isso? Quando fazemos o deslocamento e damos zoom, os rótulos dos eixos podem mudar e causar um deslocamento na posição do título do eixo. Ao definir um valor mínimo de extensão, podemos reduzir movimentos que podem causar distração. *Tente alterar o valor de `minExtent`, por exemplo definindo-o como zero, e então dê zoom para ver o que acontece quando rótulos de eixos mais longos entram na visualização.*

O Altair também inclui uma forma abreviada para adicionar deslocamento e zoom a um gráfico. Ao invés de criar diretamente uma seleção, você pode chamar `.interactive()` para fazer o Altair gerar automaticamente uma seleção de intervalo limitada às escalas do gráfico.

In [ ]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # usa o parâmetro minExtent para estabilizar o título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
).interactive()

alt.Chart(...)

Por padrão, as vinculações de escala para seleções incluem os canais de codificação `x` e `y`. E se quisessemos limitar o deslocamento e o zoom a uma única dimensão? Podemos usar `encodings=['x']` para restringir a seleção apenas ao canal `x`:

In [ ]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)),  # Usa minExtent para estabilizar o título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).add_params(
    alt.selection_interval(bind='scales', encodings=['x'])  # Usando add_params em vez de add_selection
).properties(
    width=600,
    height=400
)

alt.Chart(...)

*Ao dar zoom ao longo de um único eixo, a forma dos dados visualizados pode mudar, afetando potencialmente nossa percepção de relacionamentos nos dados. [Escolher uma proporção adequada](http://vis.stanford.edu/papers/arclength-banking) é uma consideração importante no design de visualizações!*